## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-13-10-53-35 +0000,bbc,Scottish government aiming to issue first bond...,https://www.bbc.com/news/articles/cj97dw17k3do...
1,2025-11-13-10-50-35 +0000,nyt,Former Syrian Security Official Indicted in Au...,https://www.nytimes.com/2025/11/12/world/europ...
2,2025-11-13-10-45-00 +0000,wsj,JD.com Profit Slumps as Heavy Food-Delivery Sp...,https://www.wsj.com/business/earnings/jd-com-p...
3,2025-11-13-10-41-00 +0000,wsj,"Eurozone Industry Ekes Out Growth, Dragged by ...",https://www.wsj.com/economy/eurozone-industry-...
4,2025-11-13-10-30-00 +0000,wsj,"Prosecutor in James Comey, Letitia James Cases...",https://www.wsj.com/us-news/law/prosecutor-in-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2322/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
137,trump,48
112,epstein,25
148,shutdown,25
1,government,18
38,new,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
140,2025-11-13-01-03-00 +0000,wsj,Democrats released new emails from Jeffrey Eps...,https://www.wsj.com/politics/policy/democrats-...,174
79,2025-11-13-06-05-00 +0000,wsj,President Trump signed a spending package to r...,https://www.wsj.com/politics/policy/house-spen...,141
135,2025-11-13-01-27-32 +0000,nypost,"Trump signs funding bill to end historic, 43-d...",https://nypost.com/2025/11/12/us-news/house-vo...,131
97,2025-11-13-04-11-30 +0000,wapo,Longest government shutdown in U.S. history en...,https://www.washingtonpost.com/business/2025/1...,126
129,2025-11-13-01-42-00 +0000,wsj,The turning point in the government’s longest ...,https://www.wsj.com/politics/policy/government...,126


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
140,174,2025-11-13-01-03-00 +0000,wsj,Democrats released new emails from Jeffrey Eps...,https://www.wsj.com/politics/policy/democrats-...
79,93,2025-11-13-06-05-00 +0000,wsj,President Trump signed a spending package to r...,https://www.wsj.com/politics/policy/house-spen...
227,52,2025-11-12-20-09-00 +0000,wsj,Atlanta Fed President Raphael Bostic will reti...,https://www.wsj.com/economy/central-banking/at...
240,40,2025-11-12-19-21-00 +0000,wsj,The Bank of Canada cut its main interest rate ...,https://www.wsj.com/articles/bank-of-canada-of...
362,38,2025-11-12-11-00-00 +0000,latimes,Barabak: He's loud. He's obnoxious. And Kamala...,https://www.latimes.com/politics/story/2025-11...
201,37,2025-11-12-21-19-00 +0000,wsj,Government reports on inflation and the labor ...,https://www.wsj.com/politics/policy/white-hous...
97,35,2025-11-13-04-11-30 +0000,wapo,Longest government shutdown in U.S. history en...,https://www.washingtonpost.com/business/2025/1...
308,34,2025-11-12-15-26-19 +0000,bbc,Wife and son of Gabon's ousted leader given 20...,https://www.bbc.com/news/articles/c797yg3pw7wo...
178,34,2025-11-12-22-16-49 +0000,nypost,Alexander brothers ordered to face sex traffic...,https://nypost.com/2025/11/12/business/luxury-...
55,32,2025-11-13-09-00-00 +0000,cbc,"After 2 years of war, Gaza electric company ta...",https://www.cbc.ca/news/world/gaza-power-elect...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
